In [1]:
from collections import defaultdict

load("core_utils.py")


# First pass - search for values of n that admit a decomposition

In [ ]:
existence_values = []

# We focus on odd n, then use Lemma 3.3 to check for 2n
# Note that inexistence for doubly even dimension is proved in [Theorem 3.4, And+23]
for n in range(3, 250, 2):
    en = (1<<n)-1

    factors = factor(en)
    pm1_divisors = [divisors(p[0]-1) for p in factors]

    found = False

    # We can focus on candidates of the form d = 2^i + 1, with i even [Corollary 3.1]
    # Note that this means that d is invertibile because i is even and n is odd
    for i in range(2, n, 2):
        d = (1<<i) + 1

        # Sieve with the Jacobi Symbol (i.e. even order)
        if not any(jacobi_symbol(d, p[0]) == -1 for p in factors):
            continue

        # Already find the half order, that is the order of d^2
        e = order(d^2, factors, pm1_divisors)

        # Check that the dyadic valuation is exactly 1
        if e % 2 == 0:
            continue

        if pow(d, e, en) == en - 1:
            if not found:
                existence_values.append(n)
                found = True

            # Check the conditions of Lemma 3.3
            # We use the same checks we used for 2^n-1, but apply them to 2^n+1
            # Invertibility is ensured by Lemma 2.2
            enp1 = en + 2

            factors_enp1 = factor(enp1)
            pm1_divisors_enp1 = [divisors(p[0]-1) for p in factors_enp1]

            if not any(jacobi_symbol(d, p[0]) == -1 for p in factors_enp1):
                continue

            e = order(d^2, factors_enp1, pm1_divisors_enp1)
            
            if pow(d, e, enp1) == en + 1:
                existence_values.append(2 * n)
                break


existence_values.sort()

existence_values


[3,
 5,
 6,
 7,
 10,
 13,
 14,
 15,
 17,
 19,
 23,
 26,
 30,
 31,
 34,
 38,
 43,
 46,
 51,
 59,
 61,
 62,
 65,
 67,
 79,
 83,
 85,
 86,
 89,
 93,
 95,
 97,
 101,
 102,
 103,
 107,
 109,
 122,
 127,
 129,
 130,
 131,
 133,
 137,
 139,
 149,
 158,
 170,
 178,
 186,
 193,
 197,
 202,
 214,
 215,
 218,
 221,
 227,
 233,
 241,
 254,
 266,
 386,
 454]

# Second pass - find shortest decomposition for the values of n that admit a decomposition

In [ ]:
try:
    _ = existence_values
except NameError:
    existence_values = [3, 5, 6, 7, 10, 13, 14, 15, 17, 19, 23, 26, 30, 31, 34, 38, 43, 46, 51, 59, 61, 62, 65, 67, 79, 83, 85, 86, 89, 93, 95, 97, 101, 102, 103, 107, 109, 122, 127, 129, 130, 131, 133, 137, 139, 149, 158, 170, 178, 186, 193, 197, 202, 214, 215, 218, 221, 227, 233, 241]

existence_values = [3, 5, 6, 7, 10, 13, 14, 15, 17, 19, 23, 26, 30, 31, 34, 38, 43, 46, 51, 59, 61, 62, 65, 67, 79, 83, 85, 86, 89, 93, 95, 97, 101, 102, 103, 107, 109, 122, 127, 129, 130, 131, 133, 137, 139, 149, 158, 170, 178, 186, 193, 197, 202, 214, 215, 218, 221, 227, 233, 241]


shortest_decompositions = defaultdict(lambda: {"d": 0, "i": 0, "j": 0, "e": float('inf')})

for n in existence_values:
    if n % 2 == 0: continue

    en = (1<<n)-1

    factors = factor(en)
    pm1_divisors = [divisors(p[0]-1) for p in factors]

    for i in range(2, n, 2):
        d = (1<<i) + 1

        # Sieve with the Jacobi Symbol (i.e. even order)
        if not any(jacobi_symbol(d, p[0]) == -1 for p in factors):
            continue

        # Already find the half order, that is the order of d^2
        e = order(d^2 % en, factors, pm1_divisors)
        
        # Second sieve on the order, that is, the half order is odd
        if e % 2 == 0:
            continue

        if pow(d, e, en) == en - 1:
            # Traverse the cyclotomic equivalence class, these are all guaranteed to
            # be valid candidates
            for j in range(n):
                d1 = (d << j) % en
                e1 = order(d1^2 % en, factors, pm1_divisors)
                
                decomposition = {"d": d1, "i": i+j, "j":j, "e": e1}

                if e1 < shortest_decompositions[n]["e"]:
                    shortest_decompositions[n] = decomposition
                
                if True:
                    assert pow(d1, e1, en) == en - 1, f"Candidate should have been valid. {d1=} {e1=} {en=}"

            # Check the conditions of Lemma 3.3
            # We use the same checks we used for 2^n-1, but apply them to 2^n+1
            # Invertibility is ensured by Lemma 2.2
            enp1 = en + 2

            factors_enp1 = factor(enp1)
            pm1_divisors_enp1 = [divisors(p[0]-1) for p in factors_enp1]

            if not any(jacobi_symbol(d, p[0]) == -1 for p in factors_enp1):
                continue

            f = order(d^2, factors_enp1, pm1_divisors_enp1)
            
            if pow(d, f, enp1) == en + 1:
                e2n = (1 << (2 * n)) - 1

                # Traverse the cyclotomic equivalence class
                # Note that here the members of the class are
                # only valid if dyadic(j) > 0
                for j in range(0, 2*n, 2):
                    d1 = (d << j)

                    e1 = order(d1^2 % en, factors, pm1_divisors)
                    f1 = order(d1^2 % enp1, factors_enp1, pm1_divisors_enp1)

                    order_d1 = lcm(e1, f1)

                    decomposition = {"d": d1, "i": i+j, "j":j, "e": order_d1}

                    if order_d1 < shortest_decompositions[2*n]["e"]:
                        shortest_decompositions[2*n] = decomposition


dict(sorted(shortest_decompositions.items()))


{3: {'d': 6, 'i': 4, 'j': 2, 'e': 1},
 5: {'d': 6, 'i': 6, 'j': 2, 'e': 3},
 6: {'d': 5, 'i': 2, 'j': 0, 'e': 3},
 7: {'d': 20, 'i': 4, 'j': 2, 'e': 3},
 10: {'d': 17, 'i': 4, 'j': 0, 'e': 15},
 13: {'d': 6144, 'i': 24, 'j': 12, 'e': 35},
 14: {'d': 5, 'i': 2, 'j': 0, 'e': 21},
 15: {'d': 16385, 'i': 14, 'j': 0, 'e': 75},
 17: {'d': 1025, 'i': 10, 'j': 0, 'e': 15},
 19: {'d': 1280, 'i': 10, 'j': 8, 'e': 1971},
 23: {'d': 257, 'i': 8, 'j': 0, 'e': 2231},
 26: {'d': 4097, 'i': 12, 'j': 0, 'e': 455},
 30: {'d': 16385, 'i': 14, 'j': 0, 'e': 75},
 31: {'d': 41943040, 'i': 25, 'j': 23, 'e': 3148803},
 34: {'d': 65, 'i': 6, 'j': 0, 'e': 21845},
 38: {'d': 5, 'i': 2, 'j': 0, 'e': 262143},
 43: {'d': 65, 'i': 6, 'j': 0, 'e': 197737005},
 46: {'d': 257, 'i': 8, 'j': 0, 'e': 135615797},
 51: {'d': 4194305, 'i': 22, 'j': 0, 'e': 30001923},
 59: {'d': 16385, 'i': 14, 'j': 0, 'e': 305327091563275},
 61: {'d': 17592186306560, 'i': 44, 'j': 18, 'e': 60001119157265},
 62: {'d': 65, 'i': 6, 'j': 0, 'e':